In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random


SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, val_data = train_data.split(random_state = random.seed(SEED))

In [15]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,max_size = MAX_VOCAB_SIZE,vectors = 'glove.6B.100d', unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [16]:
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iterator, val_iteraor, test_iterator = data.BucketIterator.splits((train_data, val_data, test_data), device = device, batch_size = BATCH_SIZE)

In [17]:
for batch in train_iterator:
    print(batch.text.size())
    break

torch.Size([932, 64])


In [145]:
import torch.nn as nn
import torch.nn.functional as F
class CNN_LSTM(nn.Module):
    def __init__(self, input_dim,embedding_dim,n_filters, filter_sizes, 
                 output_dim, hidden_dim, n_layers, bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = 1, out_channels = n_filters, kernel_size = (f, embedding_dim)) for f in filter_sizes])
        
        self.rnn = nn.LSTM(embedding_dim,hidden_dim, num_layers = n_layers,bidirectional = bidirectional, dropout = dropout)
        
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #[sent len , batch size ]
        
        text = text.permute(1,0)
        
        #[batch size, sent len]
        
        embedded = self.dropout(self.embedding(text))
        
        #[batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #[batch size, 1,sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        #conved[n] = [batch size, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        pooled = torch.stack(pooled)
        
        #pooled = [1, batch size, n_filters]
        packed_output, (hidden, cell) = self.rnn(pooled)
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:],hidden[-1,:,:]), dim = 1))
        
        #hidden = [batch size, hid_dim*num_directions]
        
        return(self.fc(hidden))

In [168]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 250
N_FILTERS = 250
FILTER_SIZES = [3, 4, 5, 6]
OUTPUT_DIM = 1
HIDDEN_DIM = 256
N_LAYERS = 4
BIDIRECTIONAL = True
DROPOUT = 0.6
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN_LSTM(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES,
                OUTPUT_DIM, HIDDEN_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [169]:
def count_parameters(model):
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [170]:
count_parameters(model)

13148277


In [171]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

In [172]:
model = model.to(device)
criterion = criterion.to(device)

In [173]:
def binary_accuracy(preds, y):
    
    rounded = torch.round(torch.sigmoid(preds))
    
    correct = (rounded == y).float()
    
    acc = correct.sum()/len(correct)
    
    return(acc)

In [174]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return(epoch_loss/len(iterator), epoch_acc/len(iterator))

In [175]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        return(epoch_loss/len(iterator), epoch_acc/len(iterator))

In [176]:
import time

def elapsed_time(start_time,end_time):
    elapsed_time = end_time-start_time
    elapsed_mins = int(elapsed_time/60)
    elapsed_secs = int(elapsed_time - elapsed_mins*60)
    
    return(elapsed_mins, elapsed_secs)

In [178]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model,train_iterator,optimizer,criterion)
    val_loss, val_acc = evaluate(model, val_iteraor, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = elapsed_time(start_time,end_time)
    
    if val_loss<best_valid_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(),'tut6-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 51s
	Train Loss: 0.567 | Train Acc: 71.06%
	 Val. Loss: 0.522 |  Val. Acc: 75.42%
Epoch: 02 | Epoch Time: 0m 51s
	Train Loss: 0.484 | Train Acc: 77.82%
	 Val. Loss: 0.456 |  Val. Acc: 80.87%
Epoch: 03 | Epoch Time: 0m 51s
	Train Loss: 0.423 | Train Acc: 81.34%
	 Val. Loss: 0.439 |  Val. Acc: 82.91%
Epoch: 04 | Epoch Time: 0m 50s
	Train Loss: 0.374 | Train Acc: 84.36%
	 Val. Loss: 0.399 |  Val. Acc: 84.88%
Epoch: 05 | Epoch Time: 0m 51s
	Train Loss: 0.344 | Train Acc: 86.02%
	 Val. Loss: 0.456 |  Val. Acc: 85.52%
Epoch: 06 | Epoch Time: 0m 51s
	Train Loss: 0.305 | Train Acc: 87.91%
	 Val. Loss: 0.376 |  Val. Acc: 85.79%
Epoch: 07 | Epoch Time: 0m 51s
	Train Loss: 0.290 | Train Acc: 88.81%
	 Val. Loss: 0.360 |  Val. Acc: 86.07%
Epoch: 08 | Epoch Time: 0m 51s
	Train Loss: 0.273 | Train Acc: 89.67%
	 Val. Loss: 0.345 |  Val. Acc: 86.98%
Epoch: 09 | Epoch Time: 0m 50s
	Train Loss: 0.260 | Train Acc: 89.90%
	 Val. Loss: 0.339 |  Val. Acc: 87.21%
Epoch: 10 | Epoch T

In [179]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.326 | Test Acc: 87.50%
